## Install required package

In [0]:
!pip install langchain==0.0.317
!pip install openai==0.27.8
!pip install PyPDF2 
!pip install faiss-cpu==1.7.4
!pip install tiktoken
!pip install pypdf panel notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-c8b4a793-105f-4a85-9c92-98063c9a1d52
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.0.1
    Not uninstalling tenacity at /databricks/python3/lib/python3.10/site-pa

## Import libraries and environment variables

In [0]:
import os
import openai
from pathlib import Path
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
#from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
#from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import panel as pn
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS
from langchain.vectorstores import Chroma

In [0]:
#Get the environment variables
os.environ["OPENAI_API_TYPE"] = "Azure"
os.environ["OPENAI_API_VERSION"] = dbutils.secrets.get("Secret-Scope", key = "azure-openai-api-version")
os.environ["OPENAI_API_BASE"] = dbutils.secrets.get("Secret-Scope", key = "azure-openai-api-base")
os.environ["OPENAI_API_KEY"] = dbutils.secrets.get("Secret-Scope", key = "open-api-key")
deployment_embed_name= dbutils.secrets.get("Secret-Scope", key = "azure-embedding-openai-deployment-name")
embed_model_name= dbutils.secrets.get("Secret-Scope", key = "azure-embedding-openai-model-name")

## Document Loading

In [0]:
loader = DirectoryLoader('/dbfs/FileStore/Benlysta', glob='**/*.pdf', loader_cls = PyPDFLoader)

## Document Pre-processing/Chunking

In [0]:
embeddings =OpenAIEmbeddings(engine=deployment_embed_name, model=embed_model_name, chunk_size=1)
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
    )
#engine is replacement of deployment as we move from langchain 0.0.200-->0.0.317

/local_disk0/.ephemeral_nfs/envs/pythonEnv-c8b4a793-105f-4a85-9c92-98063c9a1d52/lib/python3.10/site-packages/langchain/embeddings/openai.py:217: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(


In [0]:
try:
  pdfs = loader.load()
  texts = text_splitter.split_documents(pdfs)
  docsearch =FAISS.from_documents(
  texts, embeddings
  )
except FileNotFoundError:
    print('File does not exist.')



*** WARNING: max output size exceeded, skipping output. ***

ate Operation under Azure OpenAI API version [REDACTED] have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 1 second. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit..


## Creating Vectorstore

In [0]:
docsearch.save_local("/dbfs/FileStore/Rakesh/Vectorstore6")